In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.labelsize"] = 16
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
plt.rcParams["figure.titlesize"] = 18
plt.rcParams["axes.titlesize"] = 18

In [ ]:
RANDOM_SEED = 666
np.random.seed(RANDOM_SEED)

# Data Investigation

In [ ]:
bikes = pd.read_csv("../data/jay_st_bikes.csv")
bikes.head()

In [ ]:
bikes.info()

`last_reported` corresponds to the last time the station reported its info to the Citi Bike API. We see here that it's an integer. It is in "unix time" corresponding to the number of seconds since January 1st, 1970 at 00:00:00.

In [ ]:
pd.to_datetime(bikes["last_reported"], unit="s")

Actually, this is in UTC time, and we should make sure Pandas knows this.

In [ ]:
pd.to_datetime(bikes["last_reported"], unit="s", origin="unix", utc=True)

And actually, let's convert it to EST.

In [ ]:
bikes["last_reported"] = pd.to_datetime(
    bikes["last_reported"], unit="s", origin="unix", utc=True
).dt.tz_convert("US/Eastern")
bikes["last_reported"]

Ok better, but why does it start at 1970?

These are null values. Let's throw them out. 

The nice thing about having our data in proper datetime format is that we can reference it with strings in pandas.

In [ ]:
bikes = bikes[bikes["last_reported"] > "1971"]

In [ ]:
bikes

Looks like the data goes from basically 2020-yesterday. Let's save some space by dropping the `name` field. We'll also set the timestamp field as the index.

In [ ]:
bikes = bikes.set_index("last_reported")["num_bikes_available"]

In [ ]:
bikes.head()

We can now index by time.

In [ ]:
bikes.loc["2022-10-01":"2022-10-07"].plot()
None

# Basic ML

To peform ML, it helps to resample the data so that each point is evenly spaced in time.

In [ ]:
# Resample so that the index corresponds to 5 minute increments.
# We pick .last() so that we are not leaking future information into the
# past.
SAMPLE_MINUTES = 5
bikes = bikes.resample(f"{SAMPLE_MINUTES}T").last()

# Finally, we fill any missing data after the resampling using the most
# recently known data.
bikes = bikes.fillna(method="ffill")

In [ ]:
bikes.head()

Now, pick out some training and test data and create `X` and `y` matrices. We'll use all of 2021 as our training data and all data (so far) in 2022 for our test data.

In [ ]:
train_start = bikes.index.get_loc("2021-01-01 00:00:00-04:00")
train_end = bikes.index.get_loc("2022-01-01 00:00:00-04:00")

test_start = bikes.index.get_loc("2022-01-01 00:00:00-04:00")
test_end = bikes.index.get_loc("2022-10-13 00:00:00-04:00")

In [ ]:
def create_lag_features(y, start_index, end_index, num_lags):
    X = []
    # At each point in y between start and end, collect the prior `num_lags`
    # values of y to use as lag features.
    for idx in range(start_index, end_index):
        # We offset by 1 so that we are grabbing everything up until the
        # current value of y.
        X.append(y[(idx - 1) - num_lags : idx - 1])
    return np.vstack(X)

Split into training and test sets. We'll use one day's worth of data for the lag features.

In [ ]:
# One day's worth of lags.
NUM_LAGS = int(60 / SAMPLE_MINUTES * 24)

y = bikes.values

y_train = y[train_start:train_end]
y_test = y[test_start:test_end]

X_train = create_lag_features(y, train_start, train_end, NUM_LAGS)
X_test = create_lag_features(y, test_start, test_end, NUM_LAGS)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Now, let's train a simple linear regression model.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
fig, ax = plt.subplots()
start = -3_000
ax.plot(y_pred[start:], label="Predicted")
ax.plot(y_test[start - 1 : -1], label="Actual")
ax.legend()
None

In [ ]:
fig, ax = plt.subplots()
start = -50
ax.plot(y_pred[start:], label="Predicted")
ax.plot(y_test[start - 1 : -1], label="Actual")
ax.legend()
None

In [ ]:
fig, ax = plt.subplots()
ax.bar(list(range(1, len(model.coef_) + 1)[::-1]), model.coef_)
ax.set_ylabel("Coefficient Value")
ax.set_xlabel("Lag Feature")
None

Either ironically or unsurprisingly, you can get pretty good performance!

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

In [ ]:
print(f"Mean Absolute Error = {mean_absolute_error(y_test, y_pred):.3}")
print(f"R2 = {r2_score(y_test, y_pred):.3}")

Instead of predicting 5 minutes into the future, let's predict 60 minutes into the future

In [ ]:
prediction_minutes = 60
prediction_steps = int(prediction_minutes / SAMPLE_MINUTES - 1)

y_train = y[train_start + prediction_steps : train_end + prediction_steps]
y_test = y[test_start + prediction_steps : test_end + prediction_steps]

X_train = create_lag_features(y, train_start, train_end, NUM_LAGS)
X_test = create_lag_features(y, test_start, test_end, NUM_LAGS)

In [ ]:
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

While it may look good, it's mostly similar to before.

In [ ]:
fig, ax = plt.subplots()
start = -3_000
ax.plot(y_pred[start:], label="Predicted")
ax.plot(y_test[start - 1 : -1], label="Actual")
ax.legend()
None

In [ ]:
fig, ax = plt.subplots()
start = -50
ax.plot(y_pred[start:], label="Predicted")
ax.plot(y_test[start - 1 : -1], label="Actual")
ax.legend()
None

In [ ]:
fig, ax = plt.subplots()
ax.bar(list(range(1, len(model.coef_) + 1)[::-1]), model.coef_)
ax.set_ylabel("Coefficient Value")
ax.set_xlabel("Lag Feature")
None

In [ ]:
print(f"Mean Absolute Error = {mean_absolute_error(y_test, y_pred):.3}")
print(f"R2 = {r2_score(y_test, y_pred):.3}")

Lastly, let's program up a quick forecast

In [ ]:
forecast = []
forecast_start = -2500
forecast_steps = int(120 / SAMPLE_MINUTES)

X_forecast = X_test[forecast_start, :].reshape(1, -1)

for step in range(forecast_steps):
    forecast.append(model.predict(X_forecast)[0])
    X_forecast = np.roll(X_forecast, -1, axis=1)
    X_forecast[:, -1] = forecast[-1]

In [ ]:
fig, ax = plt.subplots()
start = -3_000
ax.plot(y_pred[start:], label="Predicted")
ax.plot(y_test[start - 1 : -1], label="Actual")
ax.plot(
    np.arange(forecast_start - start, forecast_start - start + forecast_steps),
    forecast,
    label="Forecast",
)
ax.legend()
None